In [1]:
import pickle
import pandas as pd
import psycopg2 as pg
import geopandas as gpd

from Data_getter.get_data import get_buildings, get_service
from Blocks_getter.blocks_getter import BlocksModel
from Provision_getter.provision_getter import ProvisionModel

/Users/mvin/Desktop/mp_dev/mp_dev/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/h8/0wmx2zx90bn60jw8zc1r4qjc0000gn/T/ipykernel_21936/2185740430.py:4: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (

In [2]:
def filter_df(polygon, blocks, gdf_prov):

    intersecting_blocks = gpd.overlay(blocks, polygon, how="intersection").drop(columns=["id_2"])
    intersecting_blocks.rename(columns={"id_1": "id"}, inplace=True)

    df_filtered = gdf_prov[gdf_prov["id"].isin(intersecting_blocks["id"])]

    return df_filtered

In [3]:
# GROUPS = ['schools', 'kindergartens', 'recreational_areas']
GROUPS = ["schools", "kindergartens"]
polygon = gpd.read_file("./output_data/polygon.geojson")

city_crs = 32636
city_id = 1
service_type = "schools"
matrix = pd.read_pickle("/Users/mvin/Desktop/mp_dev/MASTERPLAN/Data/MMG_SPB.pkl")
engine = pg.connect("dbname='city_db_final' user='postgres' host='10.32.1.107' port='5432' password='postgres'")
# buildings_gdf = get_buildings(engine=engine, city_crs=city_crs, city_id=city_id)
# service_gdf = get_service(engine=engine, city_crs=city_crs, city_id=city_id, service_type=service_type)
# blocks_gdf = BlocksModel(city_name='Санкт-Петербург', city_admin_level=5, city_crs=city_crs).get_blocks()
buildings_gdf = gpd.read_parquet("./output_data/buildings.parquet")
blocks_gdf = gpd.read_parquet("./output_data/blocks.parquet")

services_accessabilities = {"schools": 7, "kindergartens": 6.5, "recreational_areas": 30}
standard_dict = {"schools": 120, "kindergartens": 61, "recreational_areas": 12000}


for n in GROUPS:
    service_type = n
    service_gdf = get_service(engine=engine, city_crs=city_crs, city_id=city_id, service_type=service_type)

    model = ProvisionModel(
        SUB_GROUP=service_type,
        blocks=blocks_gdf,
        matrix=matrix,
        buildings=buildings_gdf,
        standard=standard_dict[service_type],
        ACCS_TIME=services_accessabilities[service_type],
        service_gdf=service_gdf,
    )

    if service_type == "schools":
        school_prov = model.run()
        df_filtered_school = filter_df(polygon, blocks_gdf, school_prov)
        school = df_filtered_school

    elif service_type == "kindergartens":
        kindergarten_prov = model.run()
        df_filtered_kindergarten = filter_df(polygon, blocks_gdf, kindergarten_prov)
        kindergarten = df_filtered_kindergarten

    # elif service_type == 'recreational_areas':
    #     greening = model.run()

/Users/mvin/Desktop/mp_dev/mp_dev/lib/python3.9/site-packages/geopandas/io/sql.py:158: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


  0%|          | 0/605 [00:00<?, ?it/s]

количество кварталов c сервисом schools: 51
количество жилых кварталов: 1135
количество кварталов всего: 1609
количество кварталов c ошибкой: 0


/Users/mvin/Desktop/mp_dev/mp_dev/lib/python3.9/site-packages/geopandas/io/sql.py:158: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


  0%|          | 0/691 [00:00<?, ?it/s]

количество кварталов c сервисом kindergartens: 29
количество жилых кварталов: 1120
количество кварталов всего: 1518
количество кварталов c ошибкой: 0


In [7]:
# school = gpd.read_parquet('./output_data/schools.parquet')
# kindergarten = gpd.read_parquet('./output_data/kindergartens.parquet')
greening = gpd.read_parquet("./output_data/recreational_areas.parquet")

In [8]:
gdf = gpd.read_parquet("./output_data/GLEB_BLOCKS_AND_BUILDINGS_INFO_2.parquet")
Hectare = 10000
intersecting_blocks = gpd.overlay(gdf, polygon, how="intersection").drop(columns=["id"])
gdf = intersecting_blocks

gdf["current_building_area"] = gdf["current_living_area"] + gdf["current_industrial_area"]
gdf_ = gdf[
    [
        "block_id",
        "area",
        "current_living_area",
        "current_industrial_area",
        "current_population",
        "current_green_area",
        "floors",
    ]
]

gdf_ = (
    gdf_.merge(school[["id", "population_unprov_schools"]], left_on="block_id", right_on="id")
    .merge(kindergarten[["id", "population_unprov_kindergartens"]], left_on="block_id", right_on="id")
    .merge(greening[["id", "population_unprov_recreational_areas"]], left_on="block_id", right_on="id")
)
gdf_.drop(["id_x", "id_y", "id"], axis=1, inplace=True)

gdf_["area"] = gdf_["area"] / Hectare
gdf_["current_living_area"] = gdf_["current_living_area"] / Hectare
gdf_["current_industrial_area"] = gdf_["current_industrial_area"] / Hectare
gdf_["current_green_area"] = gdf_["current_green_area"] / Hectare

df_sum = gdf_.sum()
df_sum["floors"] = gdf_["floors"].mean()
df_new = pd.DataFrame(df_sum).T
gdf_ = df_new

sample = gdf_[gdf_["area"] > 10].sample()
sample = sample.to_dict("records")
block = sample[0].copy()
block

{'block_id': 140932.0,
 'area': 270.7060423617665,
 'current_living_area': 162.287878887,
 'current_industrial_area': 50.056015810199995,
 'current_population': 49734.0,
 'current_green_area': 17.376400000000004,
 'floors': 3.2708333333333335,
 'population_unprov_schools': 1240.0,
 'population_unprov_kindergartens': 0.0,
 'population_unprov_recreational_areas': 14336.0}

In [9]:
from Balancer.balancer import MasterPlan

mp = MasterPlan(
    area=block["area"],
    current_living_area=block["current_living_area"],
    current_industrial_area=block["current_industrial_area"],
    current_population=block["current_population"],
    current_green_area=block["current_green_area"],
    current_unprov_schoolkids=block["population_unprov_schools"],
    current_unprov_kids=block["population_unprov_kindergartens"],
    current_unprov_green_population=block["population_unprov_recreational_areas"],
)
final = mp.optimal_solution_indicators()

/Users/mvin/Desktop/masterplanning/Balancer/balancer.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(temp_result, ignore_index=True)
/Users/mvin/Desktop/masterplanning/Balancer/balancer.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(temp_result, ignore_index=True)
/Users/mvin/Desktop/masterplanning/Balancer/balancer.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(temp_result, ignore_index=True)
/Users/mvin/Desktop/masterplanning/Balancer/balancer.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = 

In [10]:
final

{'area': 270.7060423617665,
 'population': 49825.0,
 'b': 30.0,
 'green_coef_G': 11.999999970362396,
 'living_area': 162.560878887,
 'school_area': 1.2,
 'school_capacity': 250,
 'kindergarten_area': 0,
 'kindergarten_capacity': 0,
 'green_area': 34.688799957241834,
 'G_min_capacity': 57814.66659540306,
 'G_max_capacity': 28907.33329770153,
 'green_coef_G_capacity': 28907.33336909687,
 'op_area': 0.000273,
 'parking1_area': 0.06879600000000001,
 'parking2_area': 0.066885}